In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [62]:
import os
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from talus_data_analysis.load import read_excel_from_gdrive, get_file_keys_in_bucket, get_file_size
from talus_data_analysis.save import write_df_to_s3
from dotenv import load_dotenv
import tempfile
import sqlite3
import math
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [4]:
load_dotenv()

True

In [5]:
gauth = GoogleAuth(settings_file="../settings.yaml")

In [69]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
LANDING_BUCKET = "talus-data-pipeline-landing-bucket"
DATA_FOLDER = "../data/210511_DalalLab"
S3_FOLDER_NARROW = "narrow/210511_DalalLab"
S3_FOLDER_WIDE = "wide/210511_DalalLab"
PROJECT_NAME = "Dalal Contract"

ELIB_FILE = "RESULTS-quant.elib"
peptide_protein_file = "peptide_proteins_results.csv"
peptide_protein_norm_output = "peptide_proteins_normalized.csv"
msstats_groupcompare_output = "msstats_groupcompare.csv"
comparison_matrix_file = "comparison_matrix.csv"

In [7]:
elib_conn = Elib(key=f"{S3_FOLDER}/{ELIB_FILE}", bucket=ENCYCLOPEDIA_BUCKET)

In [12]:
peptide_quants = elib_conn.execute_sql(sql="SELECT * FROM peptidequants;", use_pandas=True)        
peptide_quants = peptide_quants[["PeptideSeq", "TotalIntensity", "SourceFile", "PrecursorCharge"]]
peptide_quants["Run"] = peptide_quants["SourceFile"].apply(lambda x: x.split(".")[0].split("_")[-1])

In [13]:
peptide_to_protein = elib_conn.execute_sql(sql="SELECT * FROM peptidetoprotein;", use_pandas=True)
peptide_to_protein = peptide_to_protein[peptide_to_protein['isDecoy'] == False]
peptide_to_protein = peptide_to_protein[["PeptideSeq", "ProteinAccession"]]

In [14]:
peptide_df = pd.merge(peptide_to_protein, peptide_quants, how="right", on="PeptideSeq")

## Template DF

In [17]:
sample_df = read_excel_from_gdrive(gauth=gauth, key="MS_samples_April2021.xlsx", sheet="Sheet1", remove_unnamed=True)
sample_df = sample_df[(sample_df["Project"] == PROJECT_NAME) & (sample_df["Sample Description"] != "Pooled Sample")]
sample_df["Run"] = sample_df["Run ID"].apply(lambda x: x.split("_")[-1])
sample_df = sample_df.rename(columns={"Sample Name": "BioReplicate", "Sample Description": "Condition"})
sample_df = sample_df[["Run", "BioReplicate", "Condition", "Comparison"]]

In [18]:
sample_df

,Run,BioReplicate,Condition,Comparison
18,Run19,Dalal-1,DMSO 250mM,Control
19,Run20,Dalal-2,DMSO 250mM,Control
20,Run21,Dalal-3,Dox 250mM,DMSO 250mM
21,Run22,Dalal-4,Dox 250mM,DMSO 250mM
28,Run29,Dalal-5,DMSO 600mM,Control
29,Run30,Dalal-6,DMSO 600mM,Control
30,Run31,Dalal-7,Dox 600mM,DMSO 600mM
31,Run32,Dalal-8,Dox 600mM,DMSO 600mM


In [19]:
msstats_df = pd.merge(peptide_df, sample_df, how="outer", on="Run")

In [21]:
## Add a few required columns and rename header to match MSstats convention
msstats_df = msstats_df.drop(["Run", "Comparison"], axis=1)
msstats_df["IsotopeLabelType"] = "L"
msstats_df["FragmentIon"] = "y0"
msstats_df["ProductCharge"] = "1"
msstats_df = msstats_df.rename(columns={"PeptideSeq": "PeptideSequence",
                                        "ProteinAccession": "ProteinName",
                                        "SourceFile": "Run",
                                        "TotalIntensity": "Intensity"})

In [22]:
msstats_df.to_csv(f"{DATA_FOLDER}/{peptide_protein_file}")

In [23]:
write_df_to_s3(dataframe=msstats_df, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_file.replace('.csv', '.parquet')}", outputformat="parquet")

In [26]:
def get_comparison_matrix(df, filter_target_func=lambda x:x):
    df = df.sort_values(by="Condition")
    dmso_map = {condition: dmso for (condition, dmso) in zip(df["Condition"], df["Comparison"])}
    
    comp_lol = []
    targets = sorted(dmso_map.keys())
    comp_df_index = []
    for i, s in enumerate(targets):
        comp_list = [0 for t in targets]
        if dmso_map[s] in dmso_map:
            # make dmso 0
            comp_list[targets.index(dmso_map[s])] = -1
            # make target itself 1
            comp_list[i] = 1

            comp_lol.append(comp_list)
            comp_df_index.append(f"{filter_target_func(s)}/{dmso_map[s]}")

    comp_df = pd.DataFrame(comp_lol)
    comp_df.index = comp_df_index
    
    return comp_df

In [27]:
comp_matrix = get_comparison_matrix(df=sample_df, filter_target_func=lambda x: x.split(" - ")[0])

In [28]:
comp_matrix

,0,1,2,3
Dox 250mM/DMSO 250mM,-1,0,1,0
Dox 600mM/DMSO 600mM,0,-1,0,1


In [29]:
comp_matrix.to_csv(f"{DATA_FOLDER}/{comparison_matrix_file}")

# Run R Script (MSStats) ...

## Write msstats normalized peptide protein df to s3

In [30]:
msstats_df_norm = pd.read_csv(f"{DATA_FOLDER}/{peptide_protein_norm_output}")

In [31]:
msstats_df_norm

,PROTEIN,PEPTIDE,TRANSITION,FEATURE,LABEL,GROUP_ORIGINAL,SUBJECT_ORIGINAL,RUN,GROUP,SUBJECT,INTENSITY,SUBJECT_NESTED,ABUNDANCE,FRACTION,originalRUN,censored
0,sp|A0AVT1|UBA6_HUMAN,FISADVHGIWSR_3,y0_1,FISADVHGIWSR_3_y0_1,L,DMSO 250mM,Dalal-2,1,1,2,1.731125e+06,1.2,21.386151,1,210511_Talus_Run20.mzML,False
1,sp|A0AVT1|UBA6_HUMAN,TVFFESLER_2,y0_1,TVFFESLER_2_y0_1,L,DMSO 250mM,Dalal-2,1,1,2,1.000000e+00,1.2,0.000000,1,210511_Talus_Run20.mzML,True
2,sp|A0AVT1|UBA6_HUMAN,YVDLTVSFAPDIDGDEDLPGPPVR_3,y0_1,YVDLTVSFAPDIDGDEDLPGPPVR_3_y0_1,L,DMSO 250mM,Dalal-2,1,1,2,1.000000e+00,1.2,0.000000,1,210511_Talus_Run20.mzML,True
3,sp|A0FGR8|ESYT2_HUMAN,NLIAFSEDGSDPYVR_2,y0_1,NLIAFSEDGSDPYVR_2_y0_1,L,DMSO 250mM,Dalal-2,1,1,2,2.140198e+05,1.2,18.370257,1,210511_Talus_Run20.mzML,False
4,sp|A0FGR8|ESYT2_HUMAN,TLNPVFDQSFDFSVSLPEVQRR_3,y0_1,TLNPVFDQSFDFSVSLPEVQRR_3_y0_1,L,DMSO 250mM,Dalal-2,1,1,2,1.000000e+00,1.2,0.000000,1,210511_Talus_Run20.mzML,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113139,sp|Q9Y6X9|MORC2_HUMAN,TNIVALLQK_2,y0_1,TNIVALLQK_2_y0_1,L,Dox 600mM,Dalal-8,8,4,8,1.000000e+00,4.8,0.000000,1,210511_Talus_Run32.mzML,True
113140,sp|Q9Y6Y8|S23IP_HUMAN,GFFNIYHPLDPVAYR_3,y0_1,GFFNIYHPLDPVAYR_3_y0_1,L,Dox 600mM,Dalal-8,8,4,8,8.352691e+05,4.8,38.383978,1,210511_Talus_Run32.mzML,False
113141,sp|Q9Y6Y8|S23IP_HUMAN,RLEFPSGETIVMHNPK_3,y0_1,RLEFPSGETIVMHNPK_3_y0_1,L,Dox 600mM,Dalal-8,8,4,8,1.000000e+00,4.8,0.000000,1,210511_Talus_Run32.mzML,True
113142,sp|Q9Y6Y8|S23IP_HUMAN,SIIECVDDFR_2,y0_1,SIIECVDDFR_2_y0_1,L,Dox 600mM,Dalal-8,8,4,8,1.000000e+00,4.8,0.000000,1,210511_Talus_Run32.mzML,True


In [32]:
write_df_to_s3(dataframe=msstats_df_norm, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_norm_output.replace('.csv', '.parquet')}", outputformat="parquet")

## Write msstats groupcompare df to s3

In [33]:
msstats_groupcompare = pd.read_csv(f"{DATA_FOLDER}/{msstats_groupcompare_output}")

In [34]:
msstats_groupcompare

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
0,sp|A0AVT1|UBA6_HUMAN,Dox 250mM/DMSO 250mM,1.742978,3.309358,0.526682,4.0,0.626269,0.999793,NaN,0.250000,0.250000
1,sp|A0FGR8|ESYT2_HUMAN,Dox 250mM/DMSO 250mM,0.394675,5.140750,0.076774,4.0,0.942490,0.999793,NaN,0.166667,0.166667
2,sp|A1A4S6|RHG10_HUMAN,Dox 250mM/DMSO 250mM,-1.881733,NaN,NaN,0.0,NaN,NaN,NaN,0.500000,0.000000
3,sp|A1L0T0|ILVBL_HUMAN,Dox 250mM/DMSO 250mM,0.539866,0.242348,2.227648,3.0,0.112242,0.999793,NaN,0.000000,0.000000
4,sp|A2RRP1|NBAS_HUMAN,Dox 250mM/DMSO 250mM,0.907776,0.575056,1.578589,3.0,0.212544,0.999793,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5869,sp|Q9Y6R0|NUMBL_HUMAN,Dox 600mM/DMSO 600mM,-0.562784,1.235239,-0.455607,3.0,0.679627,0.809660,NaN,0.250000,0.000000
5870,sp|Q9Y6T7|DGKB_HUMAN,Dox 600mM/DMSO 600mM,-inf,NaN,NaN,NaN,NaN,0.000000,oneConditionMissing,0.750000,0.000000
5871,sp|Q9Y6W5|WASF2_HUMAN,Dox 600mM/DMSO 600mM,2.072669,0.921708,2.248726,3.0,0.110068,0.617825,NaN,0.250000,0.000000
5872,sp|Q9Y6X9|MORC2_HUMAN,Dox 600mM/DMSO 600mM,-inf,NaN,NaN,NaN,NaN,0.000000,oneConditionMissing,0.500000,0.000000


In [35]:
write_df_to_s3(dataframe=msstats_groupcompare, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{msstats_groupcompare_output.replace('.csv', '.parquet')}", outputformat="parquet")